In [3]:
#Imports
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import os
from pathlib import Path

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import platform
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(precision=3, suppress=True)

In [4]:
analysis = 'kernel_ridge_regression/outliers_excluded'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/final_analyses/maxima_and_minima_rois/roi_roi_correlation/max_min_and_sbc_to_vta/for_regression_no_sbc.csv'
    out_dir = os.path.join(os.path.dirname(data_path), f'{analysis}')
    #out_dir = r'path to out dir here'
    
    print('I have set pathnames in the Mac style')
    print('I will save to :', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    data_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\addbs_vta_to_memory_net_maxima\linear_regression_addbs_vta_to_memory_net_maxima\prepared_data.csv'
    out_dir = rf'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\addbs_vta_to_memory_net_maxima\linear_regression_addbs_vta_to_memory_net_maxima\{analysis}'
    #out_dir = r'path to out dir here'

    print('I have set pathnames in the Windows style')

NameError: name 'data_path' is not defined

In [5]:
variables_to_change = {}
variables_to_change['original'] = [
    '% Change from baseline (ADAS-Cog11)'
  
    ]
variables_to_change['destination'] = [
    'percent_change_adascog11'
    
    ]
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(conn_path)
try:
    change_string = ''
    for i in range (0, len(variables_to_change['original'])):
        change_string += f"variables_to_change['original'][{i}]: variables_to_change['destination'][{i}],"
    change_string = 'data_df.rename(columns={' + change_string[0:-1] + '})'
    data_df = eval(change_string)
except:
    print('no change necessary')

data_df = data_df.reset_index(drop=True)

#Select specific subgroup
# outlier_index = (data_df['percent_change_adascog11'] <= -50)
# data_df = data_df.loc[outlier_index, :]

# Remove outlier
outlier_index=[11, 47, 48, 49]
data_df = data_df.drop(index=outlier_index)
data_df.reset_index(drop=True, inplace=True)

try:    
    data_df.pop('Patient # CDR, ADAS')
    data_df.pop('Age at DOS')
    data_df.pop('Randomization Arm')
    data_df.pop('Baseline ADAS-Cog11')
    data_df.pop('Baseline CDR (sum of squares)')
    data_df.pop('% Change from baseline (CDR)')
    data_df.pop('Unnamed: 0')
    data_df.dropna(inplace=True)
    data_df.reset_index(drop=True, inplace=True)
except:
    print('none to pop')
    
#Standardize the data
preserved_df = data_df.copy()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
# data_df = data_df.apply(scaler.fit_transform(), axis=0)
data_df[data_df.columns] = scaler.fit_transform(data_df[data_df.columns])


# #move outcomes to start
# first_column = data_df.pop('percent_change_adascog11')  # Remove column and store it in a variable
# data_df.insert(0, 'percent_change_adascog11', first_column)  # Insert column at the first position

display(data_df)

,percent_change_adascog11,one,two,three,four,foive,six,seven,eight,nine,...,ferguson_memnet_lat_temporal_cortex_right,ferguson_memnet_med_occipital_cortex_left,ferguson_memnet_med_occipital_cortex_right,ferguson_memnet_sbc_left,ferguson_memnet_sbc_retrospl_jxn_left,ferguson_memnet_sbc_retrospl_jxn_right,ferguson_memnet_sbc_right,ferguson_memnet_sup_frontal_cortex_left,ferguson_memnet_sup_frontal_cortex_right,ferguson_memnet_vent_med_pfc_midline
0,0.317537,0.846794,-0.118939,0.138551,-0.396692,0.529664,0.192008,0.480663,1.333734,1.079662,...,-0.135829,-0.398002,-0.176829,-1.772983,-1.286670,-1.275899,-1.950037,-1.243011,-1.617249,-0.740133
1,0.014153,0.963678,0.226775,-0.308575,-1.207999,0.527137,-0.372326,1.038726,1.365882,0.300422,...,0.370693,-1.829220,-1.538027,-1.776681,-1.358288,-1.383558,-2.152321,-0.961905,-1.745146,-0.106741
2,-0.850871,-0.195769,-0.199492,-0.597591,-0.251778,-0.119691,-0.617859,0.214442,0.740688,0.366837,...,0.583547,-1.011215,-0.958002,-1.144401,-0.723332,-0.819008,-1.201740,-0.460884,-0.926581,-0.051722
3,-1.875980,0.598353,-0.427430,0.904001,-0.431090,0.940936,0.991073,-0.333571,1.087758,1.023415,...,-0.733199,0.614887,0.764313,-1.660015,-1.278705,-1.132572,-1.661516,-1.387363,-1.249446,-1.202406
4,0.539000,2.248830,2.500260,2.230417,2.061313,2.533355,2.118857,2.535749,1.212710,0.913066,...,-2.398471,-0.502225,0.677590,0.551432,-1.864491,-1.806953,0.823239,-1.269476,-0.739776,-1.811069
5,-0.028462,-0.655581,-0.914975,-0.574903,-0.366239,-0.361867,-0.437884,-0.790374,0.531280,0.541343,...,0.520670,-0.335413,-0.422092,-0.698384,-0.334838,-0.474739,-0.622246,-0.427632,-0.699617,-0.282814
6,0.440311,1.705071,1.683336,1.864186,1.440766,2.273978,1.958804,1.758698,1.330951,0.722790,...,-1.921094,-0.450009,0.612817,0.366988,-1.687716,-1.692582,0.495975,-1.397510,-1.090049,-1.693666
7,0.160349,1.719836,1.413477,1.980845,1.231687,2.240090,2.045675,1.350353,1.228151,0.179655,...,-1.865736,-0.150889,0.976052,0.477688,-1.477559,-1.401851,0.641348,-1.297846,-0.525766,-1.573846
8,0.134589,-0.796802,-0.311307,-1.058484,-0.349845,-0.863825,-1.097693,-0.158033,-0.297520,-0.953453,...,1.097353,-1.128413,-1.381106,0.104920,0.425844,0.293012,0.037268,0.621717,0.034862,0.885060
9,-0.966009,-0.873961,-0.363019,-0.293603,0.300346,-0.542027,-0.263395,-1.052902,-0.613424,-0.191304,...,0.322289,0.446668,-0.087163,0.449393,0.831772,0.775868,0.401427,0.528376,0.432211,0.126721


## Scale if desired

In [6]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled = scaler.fit_transform(data_df)
# data_df = pd.DataFrame(scaled)
# data_df=(data_df-data_df.mean())/data_df.std()


## Edit Dataframe to Remove Variables

In [19]:
new_df = data_df.copy()
all_indeps = ['Thal. Ant. Med. L']#, ''] #,Thal_Ant_Med_L 
columns_to_extract = ['percent_change_adascog11'] + all_indeps
data_df = new_df.loc[:, columns_to_extract]
data_df.head(2)

,percent_change_adascog11,Thal. Ant. Med. L
0,-21.428571,-3.093645
1,-36.363636,-2.200953


## Misc

In [12]:
data_df['Thal. Ant. Med. L'].hist(bins=19)
data_df['percent_change_adascog11'].quantile(.8)
# data_df['percent_change_adascog11'].quantile(.9)

KeyError: 'Thal. Ant. Med. L'

## Kernel Ridge Regression LOOCV

In [7]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge

def kernel_ridge_loo(df:pd.DataFrame, target:str, alpha:float, gamma:float, degree:int, kernel:str):
    y_pred = []
    for i in range(0, len(df[target].values.tolist())):
        # splitting dataframe into train and test set
        df_test = df.loc[[i]]
        df_train = df.drop(df.index[i])
        
        X_train = df_train.drop(target, axis=1)
        y_train = df_train[target]
        X_test = df_test.drop(target, axis=1)
        y_test = df_test[target]
        
        model = KernelRidge(kernel=kernel, alpha=alpha, gamma=gamma, degree=degree)
        model.fit(X_train, y_train)
        y_pred.append(model.predict(X_test)[0])
        
    outcome_dictionary = {}
    outcome_dictionary['actuals'] = df[target].values.tolist()
    outcome_dictionary['predictions'] = y_pred
    try:
        outcome_dictionary['loocv_prediction_r'], outcome_dictionary['loocv_prediction_p'] = pearsonr(outcome_dictionary['actuals'], outcome_dictionary['predictions'])
        outcome_dictionary['loocv_prediction_r2'] = outcome_dictionary['loocv_prediction_r']**2
        outcome_dictionary['loocv_mean_squared_error'] = np.sum( np.square( (outcome_dictionary['actuals'] - outcome_dictionary['predictions']))) / len(outcome_dictionary['predictions'])
        outcome_dictionary['loocv_root_mean_squared_error'] = np.sqrt(outcome_dictionary['loocv_mean_squared_error'])
        outcome_dictionary['lootcv_mean_absolute_error'] = np.sum(np.abs((outcome_dictionary['predictions'] - outcome_dictionary['actuals']))) / len(outcome_dictionary['predictions'])    
    except:
        print('Error generating outcome metrics')
    outcome_df = pd.DataFrame(outcome_dictionary)
    
    return outcome_df

In [8]:
#Reminder: kernels = ['additive_chi2', 'polynomial', 'poly', 'rbf', 'cosine', 'laplacian', 'precomputed', 'sigmoid', 'linear', 'chi2']
kernel = 'rbf'

#----------------------------------------------------------------USER INPUTS ABOVE----------------------------------------------------------------
loo_results = kernel_ridge_loo(df=data_df, target='percent_change_adascog11', kernel=kernel, alpha=0.001, gamma=.0001, degree=4)
# loo_df = pd.DataFrame(loo_results)
from scipy.stats import pearsonr, spearmanr

#Calculate metrics
loo_results.dropna(inplace=True)
loo_results['loocv_pearson_r'], loo_results['loocv_pearson_p'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
loo_results['loocv_r_squared'] = np.square(loo_results['loocv_pearson_r'])
_, loo_results['loocv_pearson_p_greater'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
_, loo_results['loocv_pearson_p_lesser'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='less')
_, loo_results['loocv_pearson_p_two-sided'] = spearmanr(loo_results['predictions'], loo_results['actuals'], alternative='two-sided')
loo_results['loocv_mse'] = (np.sum(np.square((loo_results['actuals']-loo_results['predictions'])))) / len(loo_results['actuals'])
loo_results['loocv_rmse'] = np.sqrt(loo_results['loocv_mse'])
loo_results['mean_absolute_error'] = np.sum(np.absolute(loo_results['predictions']-loo_results['actuals']))/len(loo_results['predictions'])

display(loo_results)

Error generating outcome metrics


,actuals,predictions,loocv_prediction_r,loocv_prediction_p,loocv_prediction_r2,loocv_pearson_r,loocv_pearson_p,loocv_r_squared,loocv_pearson_p_greater,loocv_pearson_p_lesser,loocv_pearson_p_two-sided,loocv_mse,loocv_rmse,mean_absolute_error
0,0.317537,-0.346514,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
1,0.014153,-0.627368,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
2,-0.850871,0.017863,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
3,-1.875980,-0.071976,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
4,0.539000,0.183770,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
5,-0.028462,-0.475985,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
6,0.440311,-0.188709,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
7,0.160349,0.349308,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
8,0.134589,-0.077853,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532
9,-0.966009,-0.072475,0.25425,0.088177,0.064643,0.256938,0.042376,0.066017,0.042376,0.957624,0.084752,1.00213,1.001065,0.755532


In [22]:
#Save if desired
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    try:
        loo_results.to_csv(os.path.join(out_dir, f'loocv_{kernel}_kernel_ridge_regression_{data_df.columns.values.tolist()[1:]}_statistically_significant_{loo_results["mean_absolute_error"].values.tolist[1]}.csv'))
    except:
        loo_results.to_csv(os.path.join(out_dir, f'ols_leave_one_out_full_formula.csv'))


print('Saved to: ' + out_dir)

Made directory
Saved to: /Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/yeo_pc1_r_map_conjunction_rois_to_addbs_vtas/kernel_ridge_regression/outliers_excluded


## Assess various Kernels

In [23]:
# Define the list of kernel functions
def linear_kernel(X1, X2, **kwargs):
    """Linear kernel function"""
    return np.matmul(X1, X2.transpose())

def polynomial_kernel(x1, x2, degree, c=1, **kwargs):
    return (np.dot(x1, x2) + c)**degree

def rbf_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.power(np.linalg.norm(x1 - x2), 2))

def gaussian_kernel(x1, x2, sigma, **kwargs):
    return np.exp(-np.power(np.linalg.norm(x1 - x2), 2) / (2 * sigma ** 2))

def sigmoid_kernel(x1, x2, k=1, c=1, **kwargs):
    return np.tanh(k * np.dot(x1, x2) + c)

def laplacian_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.linalg.norm(x1 - x2))

def exponential_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.linalg.norm(x1 - x2))

## Assess Transformation of a Kernel Upon the Data

In [24]:
n = new_df.shape[0]
kernel = np.zeros((n,n))
kernel = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        kernel[i,j] = rbf_kernel(data_df.iloc[i, 1], data_df.iloc[j, 1], gamma=1)#, sigma=1)# degree=1)
        
plt.hist(kernel)

NameError: name 'new_df' is not defined

## Run LOOCV

In [27]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from scipy.stats import pearsonr, spearmanr
kernel_list = ["linear_kernel", "polynomial_kernel", "rbf_kernel", "gaussian_kernel", "sigmoid_kernel", "laplacian_kernel", "exponential_kernel"] #, "exp", "log", "tanh"]
#Iterate over the kernels of interest

result_list_kernel = []; results_list_r2 = []; results_list_r = []; results_list_p = []; results_list_mse = []; results_list_rmse = []; results_list_mae = []
for kernel_string in kernel_list:

    #Generate Kernelized data
    n = data_df.shape[0]
    kernel = np.zeros((n,n))
    Y = data_df['percent_change_adascog11']
    X = data_df.drop('percent_change_adascog11', axis=1)
    sigma = 1000
    for i in range(n):
        for j in range(n):
            kernel[i,j] = eval(kernel_string)(X.iloc[i], X.iloc[j], gamma=1, sigma=1, k=1, degree=2)#, sigma=1)# degree=1)
            
    #Train a ridge regression on kernelized data
    model = KernelRidge(kernel='precomputed')
    results = model.fit(kernel[:, :], Y[:])

    results_dict = {}
    # results_dict['predictions'] = np.dot(kernel,model.dual_coef_)
    # results_dict['observations'] = Y[:]
    result_list_kernel.append(kernel_string)
    results_list_r2.append(results.score(kernel[:, :], Y[:]))
    r, p = spearmanr(np.dot(kernel,model.dual_coef_), Y[:])
    results_list_r.append(r); results_list_p.append(p)
    results_list_mse.append((np.sum(np.square((Y[:]-np.dot(kernel,model.dual_coef_))))) / len(Y[:]))
    results_list_rmse.append(np.sqrt((np.sum(np.square((Y[:]-np.dot(kernel,model.dual_coef_))))) / len(Y[:])))
    results_list_mae.append(np.sum(np.absolute(np.dot(kernel,model.dual_coef_)-Y[:]))/len(np.dot(kernel,model.dual_coef_)))
results_dict['kernel'] = result_list_kernel
results_dict['r2'] = results_list_r2; results_dict['r'] = results_list_r; results_dict['p'] = results_list_p
results_dict['mse'] = results_list_mse; results_dict['rmse'] = results_list_rmse; results_dict['mae'] = results_list_mae

kernel_comparison_kernel_ridge_results = pd.DataFrame(results_dict)
display(kernel_comparison_kernel_ridge_results)

,kernel,r2,r,p,mse,rmse,mae
0,linear_kernel,0.261245,0.592266,1.451507e-05,0.738755,0.859509,0.647880
1,polynomial_kernel,0.557160,0.791908,5.599840e-11,0.442840,0.665462,0.463918
2,rbf_kernel,0.658915,0.931294,6.453287e-21,0.341085,0.584024,0.443326
3,gaussian_kernel,0.546750,0.883804,4.109265e-16,0.453250,0.673239,0.499906
4,sigmoid_kernel,-14.285901,0.123659,4.129226e-01,15.285901,3.909719,2.487488
5,laplacian_kernel,0.678893,0.958616,1.225938e-25,0.321107,0.566663,0.427545
6,exponential_kernel,0.678893,0.958616,1.225938e-25,0.321107,0.566663,0.427545


In [26]:
#Save Results if desired
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    kernel_comparison_kernel_ridge_results.to_csv(os.path.join(out_dir, 'kernel_comparison_kernel_ridge_results.csv'))


print('Saved to: ' + out_dir)

Saved to: /Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/yeo_pc1_r_map_conjunction_rois_to_addbs_vtas/kernel_ridge_regression/outliers_excluded


## Grid Search for Optimal Hyperparameters

In [19]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pandas as pd
## Refernece
#--- kernels = ['additive_chi2', 'polynomial', 'poly', 'rbf', 'cosine', 'laplacian', 'precomputed', 'sigmoid', 'linear', 'chi2']

# Load your dataframe
df = data_df.copy()

# Assign the independent variables and dependent variable
X = df.iloc[:,1:]
y = df.iloc[:,0]

# Define the kernel ridge model
kr = KernelRidge()

# Define evaluation metrics
scoring = 'r2' #'r2' neg_root_mean_squared_error
# Define the parameter grid for the grid search
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'kernel': ['linear', 'rbf']}

# Perform leave-one-out cross-validation
loo = LeaveOneOut()

# Perform a grid search using the leave-one-out cross-validation
grid_search = GridSearchCV(kr, param_grid, scoring=scoring, cv=len(data_df))
grid_search.fit(X, y)

# The optimal hyperparameters
print("Numer of splits:", grid_search.n_splits_)
print("Optimal parameters:", grid_search.best_params_)
print("Optimal score:", grid_search.best_score_)
# print("Overall results:", grid_search.cv_results_)



Numer of splits: 46
Optimal parameters: {'alpha': 0.001, 'gamma': 0.001, 'kernel': 'linear'}
Optimal score: nan


## Perform Manual Kernalization and Manual Ridge Regression

In [171]:
n = data_df.shape[0]
kernel = np.zeros((n,n))
Y = data_df['percent_change_adascog11']
X = data_df.drop('percent_change_adascog11', axis=1)
sigma = 1000
for i in range(n):
    for j in range(n):
        kernel[i,j] = exponential_kernel(X.iloc[i], X.iloc[j], gamma=1, k=1, degree=1)#, sigma=1)# degree=1)

In [172]:
#Generate Kernelized data
n = data_df.shape[0]
kernel = np.zeros((n,n))
Y = data_df['percent_change_adascog11']
X = data_df.drop('percent_change_adascog11', axis=1)
sigma = 1000
for i in range(n):
    for j in range(n):
        kernel[i,j] = exponential_kernel(X.iloc[i], X.iloc[j], gamma=1, k=1, degree=1)#, sigma=1)# degree=1)
        
#Train a ridge regression on kernelized data
model = KernelRidge(kernel='precomputed')
results = model.fit(kernel[:, :], Y[:])

results_dict = {}
results_dict['predictions'] = np.dot(kernel,model.dual_coef_)
results_dict['observations'] = Y[:]
results_dict['r2'] = results.score(kernel[:, :], Y[:])
results_dict['pearson_r'], results_dict['pearson_p'] = pearsonr(results_dict['predictions'], results_dict['observations'])
results_dict['mse'] = (np.sum(np.square((results_dict['observations']-results_dict['predictions'])))) / len(results_dict['observations'])
results_dict['rmse'] = np.sqrt(results_dict['mse'])
results_dict['mae'] = np.sum(np.absolute(results_dict['predictions']-results_dict['observations']))/len(results_dict['predictions'])

manual_kernel_ridge_results = pd.DataFrame(results_dict)
display(manual_kernel_ridge_results)

,predictions,observations,r2,pearson_r,pearson_p,mse,rmse,mae
0,-16.122688,-21.428571,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
1,-22.497214,-36.363636,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
2,-40.327850,-78.947368,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
3,-67.613156,-129.411765,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
4,-7.522168,-10.526316,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
5,-17.679537,-38.461538,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
6,-10.293062,-15.384615,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
7,-16.557233,-29.166667,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
8,-16.005197,-30.434783,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068
9,-45.386166,-84.615385,0.607102,0.981122,6.544425e-36,918.911092,30.313546,22.075068


## Run K-Folds on Kernel Ridge (Useful for Precomputed Kernels)
### If using kernels, must make sure the folds are symmetric
### The ridge regression will expect a symmetric Kernel Matrix from training to testing

In [173]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from scipy.stats import pearsonr
import numpy as np

def kernel_ridge_kfold(df:pd.DataFrame, target:str, k:int, sigma:float):
    kf = KFold(n_splits=k, shuffle=True)
    y_pred = []
    y_test_all = []
    for train_index, test_index in kf.split(df):
        df_train, df_test = df.iloc[train_index], df.iloc[test_index]
        X_train = df_train.drop(target, axis=1)
        y_train = df_train[target]
        X_test = df_test.drop(target, axis=1)
        y_test = df_test[target]
        model = KernelRidge(kernel='rbf', gamma=1/(2*sigma**2))
        model.fit(X_train, y_train)
        y_pred.append(model.predict(X_test))
        y_test_all.append(y_test)
    y_test_all = np.concatenate(y_test_all)
    y_pred = np.concatenate(y_pred)
    R, P = pearsonr(y_pred, y_test_all)
    return R, P

R, P = kernel_ridge_kfold(df=data_df, target='percent_change_adascog11', k=2, sigma=1)#(data_df.shape[0]-1)/6)
results_df = pd.DataFrame({'R': R, 'P': P}, index=[0])
display(results_df)

save=False
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    results_df.to_csv(os.path.join(out_dir, 'kernel_ridge_regression_k_folds_results.csv'))

,R,P
0,-0.004101,0.977451
